In [1]:
# Import necessary libraries
from bs4 import BeautifulSoup
import re
import gensim
import jieba
import pandas as pd
import warnings
import string

# Ignore DeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Load dataset
df = pd.read_csv('BABYK_merged.csv', encoding="utf-8-sig")

# Load user-defined dictionary
jieba.load_userdict("../dict/dict.txt")

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/6k/lvj277mx6834z6rvr_nxth_00000gn/T/jieba.cache
Loading model cost 0.679 seconds.
Prefix dict has been built successfully.


In [2]:
# Define functions for cleaning and tokenization
def clean(sentences):
    cleaned = []
    for sent in sentences:
        # Remove unwanted characters and patterns
        sent = re.sub(r"^\d{1,2}-\d{1,2}-\d{1,2}\s\d{1,2}:\d{1,2}\(\d+\.\d+\s(?:KB|MB)\)\s*", '', sent)
        sent = sent.lower()
#         sent = re.sub("[~^0-9]", "", sent)
        sent = re.sub(r'http\S+|https\S+', '', sent)
        sent = BeautifulSoup(sent, 'html.parser').get_text(separator=' ')
        sent = sent.translate(str.maketrans('', '', string.punctuation + '，。！？；：‘’“”（）《》【】、'))
        sent = sent.replace('emoji', '').replace('已pm', '').replace('pm', '').replace('thx', '').replace('pls', '').replace('kb', '').replace('mb', '')
        sent = re.sub('\s+', ' ', sent).strip()
        cleaned.append(sent)
    return cleaned

df['clean'] = clean(df['merged'])
df.head()

,thread_id,thread_title,msg,merged,clean
0,20901785,Uniqlo初生bb衫，大概幾時會減價？,"Thx\n初生bb衫好似未見過減價\n而家好似做梗79包,有2件\n好似再平可以5x,不過唔...",Uniqlo初生bb衫，大概幾時會減價？\nThx\n初生bb衫好似未見過減價\n而家好似做...,uniqlo初生bb衫大概幾時會減價 初生bb衫好似未見過減價 而家好似做梗79包有2件 好...
1,20905473,政府生bb，有無好既私家產檢推介,"想穩定每個月一次照超聲波\n有無好醫師推介可以照到生？\n潘錫昌\n\nbtw, 除非有異樣...",政府生bb，有無好既私家產檢推介\n想穩定每個月一次照超聲波\n有無好醫師推介可以照到生？\...,政府生bb有無好既私家產檢推介 想穩定每個月一次照超聲波 有無好醫師推介可以照到生 潘錫昌 ...
2,20913532,初生bb病左，應該去邊度睇醫生?,如題，應該去睇家庭醫生，健康院定醫院門診?\n\n兒科啦\n兒科\n睇兒科好d\n兒科私家醫...,初生bb病左，應該去邊度睇醫生?\n如題，應該去睇家庭醫生，健康院定醫院門診?\n\n兒科啦...,初生bb病左應該去邊度睇醫生 如題應該去睇家庭醫生健康院定醫院門診 兒科啦 兒科 睇兒科好d...
3,20918024,開刀生b餵人奶,睇過好多post，無論開刀定順產都要過幾日先上奶，想問下大家係醫院照同姑娘講餵人奶？定係醫院...,開刀生b餵人奶\n睇過好多post，無論開刀定順產都要過幾日先上奶，想問下大家係醫院照同姑娘...,開刀生b餵人奶 睇過好多post無論開刀定順產都要過幾日先上奶想問下大家係醫院照同姑娘講餵人...
4,20925368,有無人婚後要同6299住然後生bb？入黎分享下,結婚之後住在老公屋企大半年62已退休 99長期家庭主婦\n二人各自一間房\n我同老公一間房\...,有無人婚後要同6299住然後生bb？入黎分享下\n結婚之後住在老公屋企大半年62已退休 99...,有無人婚後要同6299住然後生bb入黎分享下 結婚之後住在老公屋企大半年62已退休 99長期...


In [3]:
def segment(sentence):
    # Segment Chinese text and remove stopwords
    stopwords = [line.strip() for line in open('../dict/canton.txt', 'r', encoding='utf-8').readlines()]
    outstr = ''
    for word in jieba.cut(sentence.strip()):
        if word not in stopwords and (len(word.strip()) > 0) and (word >= '\u4e00' and word <= '\u9fa5'):
            outstr += word + " "
    return outstr

df['tokenz'] = [segment(sent) for sent in df['clean']]
df.head()

,thread_id,thread_title,msg,merged,clean,tokenz
0,20901785,Uniqlo初生bb衫，大概幾時會減價？,"Thx\n初生bb衫好似未見過減價\n而家好似做梗79包,有2件\n好似再平可以5x,不過唔...",Uniqlo初生bb衫，大概幾時會減價？\nThx\n初生bb衫好似未見過減價\n而家好似做...,uniqlo初生bb衫大概幾時會減價 初生bb衫好似未見過減價 而家好似做梗79包有2件 好...,初生 衫 大概 時會 減價 初生 衫 見過 減價 做梗 包有 件 再平 之前 感謝祭 買過 ...
1,20905473,政府生bb，有無好既私家產檢推介,"想穩定每個月一次照超聲波\n有無好醫師推介可以照到生？\n潘錫昌\n\nbtw, 除非有異樣...",政府生bb，有無好既私家產檢推介\n想穩定每個月一次照超聲波\n有無好醫師推介可以照到生？\...,政府生bb有無好既私家產檢推介 想穩定每個月一次照超聲波 有無好醫師推介可以照到生 潘錫昌 ...,政府 生 有無 私家 產檢 推介 穩定 每個 一次 超聲波 有無 醫師 推介 照到 生 潘錫...
2,20913532,初生bb病左，應該去邊度睇醫生?,如題，應該去睇家庭醫生，健康院定醫院門診?\n\n兒科啦\n兒科\n睇兒科好d\n兒科私家醫...,初生bb病左，應該去邊度睇醫生?\n如題，應該去睇家庭醫生，健康院定醫院門診?\n\n兒科啦...,初生bb病左應該去邊度睇醫生 如題應該去睇家庭醫生健康院定醫院門診 兒科啦 兒科 睇兒科好d...,初生 病左 去邊度 睇醫生 如題 家庭醫生 健康 院定 醫院 門診 兒科 兒科 兒科 兒科 ...
3,20918024,開刀生b餵人奶,睇過好多post，無論開刀定順產都要過幾日先上奶，想問下大家係醫院照同姑娘講餵人奶？定係醫院...,開刀生b餵人奶\n睇過好多post，無論開刀定順產都要過幾日先上奶，想問下大家係醫院照同姑娘...,開刀生b餵人奶 睇過好多post無論開刀定順產都要過幾日先上奶想問下大家係醫院照同姑娘講餵人...,開刀 生 餵人奶 睇過 多p 開刀 定 順產 都要 過幾日 先上 奶 想問 醫院 照同 姑娘...
4,20925368,有無人婚後要同6299住然後生bb？入黎分享下,結婚之後住在老公屋企大半年62已退休 99長期家庭主婦\n二人各自一間房\n我同老公一間房\...,有無人婚後要同6299住然後生bb？入黎分享下\n結婚之後住在老公屋企大半年62已退休 99...,有無人婚後要同6299住然後生bb入黎分享下 結婚之後住在老公屋企大半年62已退休 99長期...,無人 婚後 要同 住 生 入黎 分享下 結婚 住在 老公 屋企 大半年 已退休 長期 家庭主...


In [4]:
# Define a function to process text data
def process(texts):
    # Build bigram and trigram models and apply to tokenized text
    bigram = gensim.models.Phrases(texts, min_count=10, threshold=10)
    trigram = gensim.models.Phrases(bigram[texts], min_count=10, threshold=10)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    # Load stopwords and Cantonese-specific words
    stop_words = [line.strip() for line in open('../dict/stopwords.txt', 'r', encoding='UTF-8').readlines()]
    canton = [line.strip() for line in open('../dict/canton.txt', 'r', encoding='UTF-8').readlines()]

    # Define a function to remove stopwords, parts of speech, and Cantonese-specific words
    def remove_words(doc):
        return [word for word in gensim.utils.simple_preprocess(str(doc), min_len=2, deacc=True)
                if word not in stop_words and word not in canton]

    # Apply processing steps to tokenized text
    texts = [[word for word in trigram_mod[bigram_mod[remove_words(doc)]]] for doc in texts]
    texts = [[word for sent in doc for word in sent.split() if not re.match(r'[^\w\s]', word)] for doc in texts]
    return texts

# Apply the processing function to the 'tokenz' column of a DataFrame
df['tokenz'] = process(df['tokenz'])

# Define a function to split sentences into individual words and remove punctuation
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = re.sub(r'[^\w\s]', '', sent)  # remove punctuations
        yield sent

# Convert the 'tokenz' column to a list of strings
df['tokenz'] = df['tokenz'].astype(str)

# Convert the 'tokenz' column into a list of lists of individual words
msg_space = df.tokenz.values.tolist()
msg_space_words = list(sent_to_words_space(msg_space))

# Assign the list of lists of individual words back to the 'tokenz' column
df['tokenz'] = msg_space_words

# Print the final DataFrame
df.head()

,thread_id,thread_title,msg,merged,clean,tokenz
0,20901785,Uniqlo初生bb衫，大概幾時會減價？,"Thx\n初生bb衫好似未見過減價\n而家好似做梗79包,有2件\n好似再平可以5x,不過唔...",Uniqlo初生bb衫，大概幾時會減價？\nThx\n初生bb衫好似未見過減價\n而家好似做...,uniqlo初生bb衫大概幾時會減價 初生bb衫好似未見過減價 而家好似做梗79包有2件 好...,初生 時會 減價 初生 見過 減價 做梗 包有 再平 感謝祭 買過 兩件 無減 要到 新款 ...
1,20905473,政府生bb，有無好既私家產檢推介,"想穩定每個月一次照超聲波\n有無好醫師推介可以照到生？\n潘錫昌\n\nbtw, 除非有異樣...",政府生bb，有無好既私家產檢推介\n想穩定每個月一次照超聲波\n有無好醫師推介可以照到生？\...,政府生bb有無好既私家產檢推介 想穩定每個月一次照超聲波 有無好醫師推介可以照到生 潘錫昌 ...,政府 私家 產檢 推介 穩定 超聲波 醫師 推介 照到 潘錫昌 異樣 實無必要 照架 慳返 ...
2,20913532,初生bb病左，應該去邊度睇醫生?,如題，應該去睇家庭醫生，健康院定醫院門診?\n\n兒科啦\n兒科\n睇兒科好d\n兒科私家醫...,初生bb病左，應該去邊度睇醫生?\n如題，應該去睇家庭醫生，健康院定醫院門診?\n\n兒科啦...,初生bb病左應該去邊度睇醫生 如題應該去睇家庭醫生健康院定醫院門診 兒科啦 兒科 睇兒科好d...,初生 病左 去邊度 睇醫生 如題 家庭醫生 健康 院定 醫院 門診 兒科 兒科 兒科 兒科 ...
3,20918024,開刀生b餵人奶,睇過好多post，無論開刀定順產都要過幾日先上奶，想問下大家係醫院照同姑娘講餵人奶？定係醫院...,開刀生b餵人奶\n睇過好多post，無論開刀定順產都要過幾日先上奶，想問下大家係醫院照同姑娘...,開刀生b餵人奶 睇過好多post無論開刀定順產都要過幾日先上奶想問下大家係醫院照同姑娘講餵人...,開刀 餵人奶 睇過 多p 開刀 順產 過幾日 先上 想問 醫院 照同 姑娘 餵人奶 醫院 食...
4,20925368,有無人婚後要同6299住然後生bb？入黎分享下,結婚之後住在老公屋企大半年62已退休 99長期家庭主婦\n二人各自一間房\n我同老公一間房\...,有無人婚後要同6299住然後生bb？入黎分享下\n結婚之後住在老公屋企大半年62已退休 99...,有無人婚後要同6299住然後生bb入黎分享下 結婚之後住在老公屋企大半年62已退休 99長期...,婚後 分享下 結婚 住在 老公 屋企 大半年 已退休 長期 家庭主婦 二人 我同 老公 結婚...


In [5]:
df.to_csv("token.csv", encoding='utf-8-sig', index=False)